In [1]:
import os
import data_analyser
import copy
import itertools
from collections import defaultdict
from operator import itemgetter

In [2]:
os.chdir('../../data')
data = data_analyser.DataAnalyser('article_sample.csv').data

In [3]:
data.head(20)

,item_id,item_name,user_id,user_name,user_editcount,user_registration,user_group,comment,comment_simplified,operation,category,rev_id,rev_len,rev_timestamp,itemquality_prediction,itemquality_A,itemquality_B,itemquality_C,itemquality_D,itemquality_E
0,114,Q17,13,Aude,24410,20121026201257,rollbacker,"Created page with ""country""",Created page,create,page,22,154,20121029172022,E,0.0010437453706178329,0.0013213738644190709,0.004874046198066724,0.029838449436460576,0.9629223851304359
1,114,Q17,13,Aude,24410,20121026201257,rollbacker,/* wbsetsitelink-set:1|enwiki */ Japan,wbsetsitelink-set,set,sitelink,27,179,20121029173026,E,0.0011032307793832803,0.0013989387002275142,0.00433084757551961,0.023133292681822126,0.9700336902630474
2,114,Q17,13,Aude,24410,20121026201257,rollbacker,/* wbsetsitelink-set:1|arwiki */ اليابان,wbsetsitelink-set,set,sitelink,30,214,20121029173106,E,0.0010909342611774026,0.0014000187429346702,0.00428269208073276,0.021864520024165817,0.9713618348909894
3,114,Q17,,,,,,/* wbsetsitelink-set:1|uzwiki */ Yaponiya,wbsetsitelink-set,set,sitelink,58,242,20121029174425,E,0.000902900660973239,0.0011585526280418874,0.003645742238393211,0.015292746100254247,0.9790000583723374
4,114,Q17,3102,Katie Filbert (WMDE),33,20121029114508,,/* wbsetlabel-set:1|en-ca */ Japan,wbsetlabel-set,set,label,72,266,20121029180539,E,0.0008877025715603948,0.001138694070900726,0.003718458603799239,0.016365114955746204,0.9778900297979936
5,114,Q17,3244,Jens Ohlig,1497,20121029180604,,/* wbsetsitelink-set:1|kowiki */ 일본,wbsetsitelink-set,set,sitelink,74,292,20121029180827,E,0.000879492717290262,0.0011255984091946072,0.003631798400434603,0.01627484352894607,0.9780882669441344
6,114,Q17,3244,Jens Ohlig,1497,20121029180604,,/* wbsetsitelink-set:1|jawiki */ 日本,wbsetsitelink-set,set,sitelink,75,318,20121029180901,E,0.0008754584375070944,0.0011295911814768447,0.0036918159147153987,0.014200818693394767,0.9801023157729056
7,114,Q17,3248,Casual,533,20121029180905,,/* wbsetsitelink-set:1|ruwiki */ Япония,wbsetsitelink-set,set,sitelink,77,351,20121029180946,E,0.000879502211585878,0.0013579041412608572,0.003690612245433153,0.01428921543254472,0.9797827659691755
8,114,Q17,155,Jeblad,7450,20121026234905,rollbacker,/* wbsetsitelink-set:1|sewiki */ Japána,wbsetsitelink-set,set,sitelink,89,378,20121029181307,E,0.0008794398025058953,0.001361917435200322,0.0037572002597132856,0.01428820147626687,0.9797132410263135
9,114,Q17,155,Jeblad,7450,20121026234905,rollbacker,/* wbsetlabel-set:1|se */ Japána,wbsetlabel-set,set,label,90,401,20121029181313,E,0.001206064609086406,0.0018588843500143648,0.00542604117857858,0.028415724283265383,0.9630932855790553


## Prepare Data

In [4]:
df = data[['item_name', 'user_name', 'comment_simplified']].head(20000)
grouped = df.groupby(['item_name', 'user_name'])

In [5]:
""" 
Created a list of itemlists
"""
def createSequences(grouped_data):
    sequences = []
    for name, group in grouped_data:
        event = []
        for user in grouped.get_group(name)['user_name']:
            activity = grouped.get_group(name)['comment_simplified'].unique().tolist()
            event.append(activity)
        sequences.append(event)
    return sequences

In [6]:
"""
This is a simple recursive method that checks if subsequence is a subSequence of mainSequence
"""
def isSubsequence(mainSequence, subSequence):
    subSequenceClone = list(subSequence) # clone the sequence, because we will alter it
    return isSubsequenceRecursive(mainSequence, subSequenceClone) #start recursion


"""
Function for the recursive call of isSubsequence, not intended for external calls
"""
def isSubsequenceRecursive(mainSequence, subSequenceClone, start=0):
    # Check if empty: End of recursion, all itemsets have been found
    if (not subSequenceClone):
        return True
    # retrieves element of the subsequence and removes is from subsequence 
    firstElem = set(subSequenceClone.pop(0))
    # Search for the first itemset...
    for i in range(start, len(mainSequence)):
        if (set(mainSequence[i]).issuperset(firstElem)):
            # and recurse
            return isSubsequenceRecursive(mainSequence, subSequenceClone, i + 1)
    return False


"""
Computes the length of the sequence (sum of the length of the contained itemsets)
"""
def sequenceLength(sequence):
    return sum(len(i) for i in sequence)


"""
Computes the support of a sequence in a dataset
"""
def countSupport (dataset, candidateSequence):
    return sum(1 for seq in dataset if isSubsequence(seq, candidateSequence)) 

In [7]:
"""
Generates one candidate of length k from two candidates of length (k-1) as used in the AprioriAll algorithm
"""
def generateCandidatesForPair(cand1, cand2):
    cand1Clone = copy.deepcopy(cand1)
    cand2Clone = copy.deepcopy(cand2)
    # drop the leftmost item from cand1:
    if (len (cand1[0]) == 1):
        cand1Clone.pop(0)
    else:
        cand1Clone[0] = cand1Clone[0][1:]
    # drop the rightmost item from cand2:
    if (len (cand2[-1]) == 1):
        cand2Clone.pop(-1)
    else:
        cand2Clone[-1] = cand2Clone[-1][:-1]
    
    # if the result is not the same, then we dont need to join
    if not cand1Clone == cand2Clone:
        return []
    else:
        newCandidate = copy.deepcopy(cand1)
        if (len (cand2[-1]) == 1):
            newCandidate.append(cand2[-1])
        else:
            newCandidate [-1].extend(cand2[-1][-1])
        return newCandidate
    

"""
Generates the set of candidates of length k from the set of frequent sequences with length (k-1)
"""
def generateCandidates(lastLevelCandidates):
    k = sequenceLength(lastLevelCandidates[0]) + 1
    if (k == 2):
        flatShortCandidates = [item for sublist2 in lastLevelCandidates for sublist1 in sublist2 for item in sublist1]
        result = [[[a, b]] for a in flatShortCandidates for b in flatShortCandidates if b > a]
        result.extend([[[a], [b]] for a in flatShortCandidates for b in flatShortCandidates])
        return result
    else:
        candidates = []
        for i in range(0, len(lastLevelCandidates)):
            for j in range(0, len(lastLevelCandidates)):
                newCand = generateCandidatesForPair(lastLevelCandidates[i], lastLevelCandidates[j])
                if (not newCand == []):
                    candidates.append(newCand)
        candidates.sort()
        return candidates
    
    
"""
Computes all direct subsequence for a given sequence.
A direct subsequence is any sequence that originates from deleting exactly one item from any event in the original sequence.
"""
def generateDirectSubsequences(sequence):
    result = []
    for i, itemset in enumerate(sequence):
        if (len(itemset) == 1):
            sequenceClone = copy.deepcopy(sequence)
            sequenceClone.pop(i)
            result.append(sequenceClone)
        else:
            for j in range(len(itemset)):
                sequenceClone = copy.deepcopy(sequence)
                sequenceClone[i].pop(j)
                result.append(sequenceClone)
    return result


"""
Prunes the set of candidates generated for length k given all frequent sequence of level (k-1), as done in AprioriAll
"""
def pruneCandidates(candidatesLastLevel, candidatesGenerated):
    return [cand for cand in candidatesGenerated if all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]

## Apriori

In [8]:
"""
The AprioriAll algorithm. Computes the frequent sequences in a seqeunce dataset for a given minSupport

Args:
    dataset: A list of sequences, for which the frequent (sub-)sequences are computed
    minSupport: The minimum support that makes a sequence frequent
    verbose: If true, additional information on the mining process is printed (i.e., candidates on each level)
Returns:
    A list of tuples (s, c), where s is a frequent sequence, and c is the count for that sequence
"""
def apriori(dataset, minSupport, verbose=False):
    global numberOfCountingOperations
    numberOfCountingOperations = 0
    Overall = []
    itemsInDataset = sorted(set ([item for sublist1 in dataset for sublist2 in sublist1 for item in sublist2]))
    singleItemSequences = [[[item]] for item in itemsInDataset]
    singleItemCounts = [(i, countSupport(dataset, i)) for i in singleItemSequences if countSupport(dataset, i) >= minSupport]
    Overall.append(singleItemCounts)
    print ("Result, lvl 1: " + str(Overall[0]))
    k = 1
    while (True):
        if not Overall [k - 1]:
            break
        # 1. Candidate generation
        candidatesLastLevel = [x[0] for x in Overall[k - 1]]
        candidatesGenerated = generateCandidates (candidatesLastLevel)
        # 2. Candidate pruning (using a "containsall" subsequences)
        candidatesPruned = [cand for cand in candidatesGenerated if all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]
        # 3. Candidate checking
        candidatesCounts = [(i, countSupport(dataset, i)) for i in candidatesPruned]
        resultLvl = [(i, count) for (i, count) in candidatesCounts if (count >= minSupport)]
        Overall.append(resultLvl)
        k = k + 1
    # "flatten" Overall
    Overall = Overall [:-1]
    Overall = [item for sublist in Overall for item in sublist]
    return Overall

In [9]:
dataset = createSequences(grouped)
apriori(dataset, 50, verbose=False)

Result, lvl 1: [([['']], 181), ([['Created page']], 208), ([['wbeditentity']], 79), ([['wbeditentity-update']], 61), ([['wbsetaliases-add-remove']], 438), ([['wbsetdescription-set']], 1735), ([['wbsetentity']], 535), ([['wbsetlabel-set']], 1099), ([['wbsetsitelink-set']], 736)]


[([['']], 181),
 ([['Created page']], 208),
 ([['wbeditentity']], 79),
 ([['wbeditentity-update']], 61),
 ([['wbsetaliases-add-remove']], 438),
 ([['wbsetdescription-set']], 1735),
 ([['wbsetentity']], 535),
 ([['wbsetlabel-set']], 1099),
 ([['wbsetsitelink-set']], 736),
 ([['', 'wbsetdescription-set']], 57),
 ([['Created page', 'wbsetdescription-set']], 54),
 ([['Created page', 'wbsetsitelink-set']], 206),
 ([['wbsetaliases-add-remove', 'wbsetdescription-set']], 286),
 ([['wbsetaliases-add-remove', 'wbsetlabel-set']], 172),
 ([['wbsetaliases-add-remove', 'wbsetsitelink-set']], 88),
 ([['wbsetdescription-set', 'wbsetentity']], 145),
 ([['wbsetdescription-set', 'wbsetlabel-set']], 709),
 ([['wbsetdescription-set', 'wbsetsitelink-set']], 222),
 ([['wbsetlabel-set', 'wbsetsitelink-set']], 246),
 ([[''], ['']], 138),
 ([[''], ['wbsetdescription-set']], 57),
 ([['Created page'], ['Created page']], 206),
 ([['Created page'], ['wbsetdescription-set']], 54),
 ([['Created page'], ['wbsetsitelin